## Reading in data available through the Planteome database
The purpose of this notebook is to read in and do a preliminary survey of the data related to text and ontology annotations of text that is available through TAIR. The datasets need to be organized and also restructured into a standard format that will allow it be combined with datasets from other resources. This notebook takes the following input files that were downloaded from the TAIR and produces a set of files that have standard columns, which are listed and described below.

### Files read
```
plant-data/databases/planteome/biological_process.txt
plant-data/databases/planteome/cellular_component.txt
plant-data/databases/planteome/molecular_function.txt
plant-data/databases/planteome/plant_anatomical_entity.txt
plant-data/databases/planteome/plant_structure_development_stage.txt
plant-data/databases/planteome/quality.txt
```

### Files created
```
plant-data/reshaped_data/planteome_curated_annotations.csv
```

### Columns in the created files
* **species_name**: String is the name of the species.
* **species_code**: String identifier for the species, uses the 3-letter codes from KEGG.
* **unique_gene_identifiers**: Pipe delimited list of gene identifers, names, models, etc that uniquely refer to this gene.
* **other_gene_identifiers**: Same as the previous, but may not uniquely refer to a given gene.
* **gene_models**: Pipe delimited list of gene model names, subset of unique_gene_identifiers.
* **text_unprocessed**: A free text field for any descriptions of phenotyes associated with this gene.
* **annotations**: Pipe delimited list of gene ontology term identifiers.
* **reference_name**: String naming the database or paper that was the source for this data.
* **reference_link**: The link to the reference resource if applicable.
* **reference_file**: The specific name of the file from which this data comes if applicable.

In [64]:
import sys
import os
import re
import warnings
import pandas as pd
import numpy as np
import itertools
import re

sys.path.append("../utils")
from constants import EVIDENCE_CODES

OUTPUT_DIR = "../reshaped_data"
warnings.simplefilter('ignore')
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [65]:
# Creating a list of lambdas for finding gene model strings.
gene_model_patterns = []
gene_model_patterns.append(re.compile("grmzm.+"))
gene_model_patterns.append(re.compile("zm[0-9]+d[0-9]+"))
gene_model_patterns.append(re.compile("solyc[0-9]+g[0-9]+"))
gene_model_patterns.append(re.compile("at[0-9]{1}g[0-9]+"))
gene_model_patterns.append(re.compile("medtr[0-9]{1}g[0-9]+"))
gene_model_patterns.append(re.compile("os[0-9]+g[0-9]+"))
gene_model_patterns.append(re.compile("loc_os[0-9]+g[0-9]+"))
is_gene_model = lambda s: any([bool(pattern.match(s.lower())) for pattern in gene_model_patterns])

In [80]:
# Columns that should be in the final reshaped files.
reshaped_columns = [
 "species_name",
 "species_code",
 "unique_gene_identifiers", 
 "other_gene_identifiers", 
 "gene_models", 
 "text_unprocessed", 
 "annotations", 
 "reference_name",
 "reference_link",
 "reference_file"]

# The files that were obtained through the Planteome browser queries.
planteome_annotation_filepaths = [
    "../databases/planteome/biological_process.txt",
    "../databases/planteome/cellular_component.txt",
    "../databases/planteome/molecular_function.txt",
    "../databases/planteome/plant_anatomical_entity.txt",
    "../databases/planteome/plant_structure_development_stage.txt",
    "../databases/planteome/quality.txt"
]
# The fields that were included for each of those downloaded files.
columns = [
    "bioentity",
    "bioentity_name",
    "type",
    "bioentity_label",
    "annotation_class",
    "aspect",
    "annotation_extension_json",
    "taxon",
    "evidence_type",
    "evidence_with",
    "reference",
    "assigned_by"
]
# Read in all the files and stack rows, because they all have the same formatting and fields.
dfs = [pd.read_csv(path, sep="\t", names=columns) for path in planteome_annotation_filepaths]
df = pd.concat(dfs)
df.head(20)

,bioentity,bioentity_name,type,bioentity_label,annotation_class,aspect,annotation_extension_json,taxon,evidence_type,evidence_with,reference,assigned_by
0,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
1,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0016032,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
2,GR_gene:GR:0116029,LOC_Os07g41220,gene,LOC_Os07g41220,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
3,GR_gene:GR:0136509,LOC_Os05g45110,gene,LOC_Os05g45110,GO:0008152,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
4,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0006333,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
5,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
6,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
7,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
8,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene
9,GR_gene:GR:0136500,LOC_Os05g43960,gene,LOC_Os05g43960,GO:0006355,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene


In [67]:
df.tail(20)

,bioentity,bioentity_name,type,bioentity_label,annotation_class,aspect,annotation_extension_json,taxon,evidence_type,evidence_with,reference,assigned_by
3394,SGN_gene:1418,Solyc03g083910,gene,sucr,TO:0000328,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3395,SGN_gene:1345,HQ317907,gene,sit,TO:0000207,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3396,SGN_gene:1345,HQ317907,gene,sit,TO:0000541,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3397,SGN_gene:1345,HQ317907,gene,sit,TO:0000652,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3398,SGN_gene:1345,HQ317907,gene,sit,TO:0002667,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3399,SGN_gene:1296,Solyc05g012020,gene,mc,TO:0000373,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3400,SGN_gene:1296,Solyc05g012020,gene,mc,TO:0000864,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3401,SGN_gene:1296,Solyc05g012020,gene,mc,TO:0002604,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3402,SGN_gene:1296,Solyc05g012020,gene,mc,TO:0002617,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson
3403,SGN_gene:1296,Solyc05g012020,gene,mc,TO:0002633,T,NaN,NCBITaxon:4081,IMP,NaN,PMID:25774204,Planteome:Ethan_Johnson


In [68]:
# How many of each evidence type are there in this combined?
code_quantities = {c:len([x for x in df["evidence_type"] if EVIDENCE_CODES[x] in c]) for c in list(set(EVIDENCE_CODES.values()))}
for k,v in code_quantities.items():
    print("{:25}{:8}".format(k,v))

computational                 138
high_throughput                 0
curator_statement            1118
experimental               168565
electronic_annotation           2
phylogenetics                   0
author_statement             9980


In [69]:
df.shape

(179803, 12)

In [70]:
# Retain just the annotations that we're considering high confidence.
high_confidence_categories = ["experimental","author_statement","curator_statement"]
df["high_confidence"] = df["evidence_type"].apply(lambda x: EVIDENCE_CODES[x] in high_confidence_categories)
df = df[df["high_confidence"]]
df.shape

(179663, 13)

In [71]:
# Taxon IDs are given in the file but we need to use the naming scheme used across files to make it compatible.
ncbi_taxon_ids = ["3702","3847","3880","4081","4530","4577"]
species_code_strings = ["ath","gmx","osa","mtr","sly","zma"]
species_name_strings = ["Arabidopsis","soybean","rice","Medicago","tomato","maize"]
mapping = dict(zip(ncbi_taxon_ids,species_code_strings))
df["species_code"] = df["taxon"].map(lambda x: mapping.get(x[-4:],None))
mapping = dict(zip(ncbi_taxon_ids,species_name_strings))
df["species_name"] = df["taxon"].map(lambda x: mapping.get(x[-4:],None))
df.dropna(subset=["species_code"], axis=0, inplace=True)
print(pd.unique(df["species_name"]))
print(pd.unique(df["species_code"]))

['tomato' 'Medicago' 'maize' 'Arabidopsis' 'soybean' 'rice']
['sly' 'mtr' 'zma' 'ath' 'gmx' 'osa']


In [72]:
df.shape

(176002, 15)

In [73]:
df.head(20)

,bioentity,bioentity_name,type,bioentity_label,annotation_class,aspect,annotation_extension_json,taxon,evidence_type,evidence_with,reference,assigned_by,high_confidence,species_code,species_name
0,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato
1,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0016032,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato
2,GR_gene:GR:0116029,LOC_Os07g41220,gene,LOC_Os07g41220,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato
3,GR_gene:GR:0136509,LOC_Os05g45110,gene,LOC_Os05g45110,GO:0008152,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato
4,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0006333,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato
5,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato
6,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato
7,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato
8,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato
9,GR_gene:GR:0136500,LOC_Os05g43960,gene,LOC_Os05g43960,GO:0006355,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato


In [74]:
# Formatting the gene identifier columns to be the same as the other files.
df["unique_gene_identifiers"] = df.apply(lambda row: "{}|{}".format(row["bioentity_name"],row["bioentity_label"]), axis=1)
df["other_gene_identifiers"] = ""
df.head(20)

,bioentity,bioentity_name,type,bioentity_label,annotation_class,aspect,annotation_extension_json,taxon,evidence_type,evidence_with,reference,assigned_by,high_confidence,species_code,species_name,unique_gene_identifiers,other_gene_identifiers
0,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato,LOC_Os06g40240|LOC_Os06g40240,
1,GR_gene:GR:0136511,LOC_Os06g40240,gene,LOC_Os06g40240,GO:0016032,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato,LOC_Os06g40240|LOC_Os06g40240,
2,GR_gene:GR:0116029,LOC_Os07g41220,gene,LOC_Os07g41220,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato,LOC_Os07g41220|LOC_Os07g41220,
3,GR_gene:GR:0136509,LOC_Os05g45110,gene,LOC_Os05g45110,GO:0008152,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato,LOC_Os05g45110|LOC_Os05g45110,
4,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0006333,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato,LOC_Os08g30200|LOC_Os08g30200,
5,GR_gene:GR:0136503,LOC_Os08g30200,gene,LOC_Os08g30200,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato,LOC_Os08g30200|LOC_Os08g30200,
6,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006278,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato,LOC_Os12g26444|LOC_Os12g26444,
7,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0006508,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato,LOC_Os12g26444|LOC_Os12g26444,
8,GR_gene:GR:0136502,LOC_Os12g26444,gene,LOC_Os12g26444,GO:0015074,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato,LOC_Os12g26444|LOC_Os12g26444,
9,GR_gene:GR:0136500,LOC_Os05g43960,gene,LOC_Os05g43960,GO:0006355,P,NaN,NCBITaxon:4530,IMP,NaN,GR_REF:0,Gramene,True,sly,tomato,LOC_Os05g43960|LOC_Os05g43960,


In [75]:
# Other columns that are needed.
df["gene_models"] = df["unique_gene_identifiers"].map(lambda x: "".join([s for s in x.split("|") if is_gene_model(s)]))
df["text_unprocessed"] = ""
df["annotations"] = df["annotation_class"]
df["reference_name"] = "Planteome"
df["reference_link"] = "https://planteome.org/"


aspects = ["P", "C", "F", "A", "T", "G"]
filenames = [
    "biological_process.txt",
    "cellular_component.txt",
    "molecular_function.txt",
    "plant_anatomical_entity.txt",
    "quality.txt",
    "plant_structural_development_stage.txt"]

df["reference_file"] = df["aspect"].map(dict(zip(aspects,filenames)))

In [76]:
df["ontology"] = df["annotation_class"].map(lambda x: x.split(":")[0])
pd.unique(df["ontology"])
df = df[df["ontology"].isin(["GO","PO","PATO"])]
df.shape

(172590, 24)

In [77]:
df = df[reshaped_columns]
df.head(30)

,species_name,species_code,unique_gene_identifiers,other_gene_identifiers,gene_models,text_unprocessed,annotations,reference_name,reference_link,reference_file
0,tomato,sly,LOC_Os06g40240|LOC_Os06g40240,,LOC_Os06g40240LOC_Os06g40240,,GO:0006278,Planteome,https://planteome.org/,biological_process.txt
1,tomato,sly,LOC_Os06g40240|LOC_Os06g40240,,LOC_Os06g40240LOC_Os06g40240,,GO:0016032,Planteome,https://planteome.org/,biological_process.txt
2,tomato,sly,LOC_Os07g41220|LOC_Os07g41220,,LOC_Os07g41220LOC_Os07g41220,,GO:0006508,Planteome,https://planteome.org/,biological_process.txt
3,tomato,sly,LOC_Os05g45110|LOC_Os05g45110,,LOC_Os05g45110LOC_Os05g45110,,GO:0008152,Planteome,https://planteome.org/,biological_process.txt
4,tomato,sly,LOC_Os08g30200|LOC_Os08g30200,,LOC_Os08g30200LOC_Os08g30200,,GO:0006333,Planteome,https://planteome.org/,biological_process.txt
5,tomato,sly,LOC_Os08g30200|LOC_Os08g30200,,LOC_Os08g30200LOC_Os08g30200,,GO:0015074,Planteome,https://planteome.org/,biological_process.txt
6,tomato,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0006278,Planteome,https://planteome.org/,biological_process.txt
7,tomato,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0006508,Planteome,https://planteome.org/,biological_process.txt
8,tomato,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0015074,Planteome,https://planteome.org/,biological_process.txt
9,tomato,sly,LOC_Os05g43960|LOC_Os05g43960,,LOC_Os05g43960LOC_Os05g43960,,GO:0006355,Planteome,https://planteome.org/,biological_process.txt


In [78]:
# Outputting the dataset of annotations to a csv file.
path = os.path.join(OUTPUT_DIR,"planteome_curated_annotations.csv")
df.to_csv(path, index=False)
df.head(30)

,species_name,species_code,unique_gene_identifiers,other_gene_identifiers,gene_models,text_unprocessed,annotations,reference_name,reference_link,reference_file
0,tomato,sly,LOC_Os06g40240|LOC_Os06g40240,,LOC_Os06g40240LOC_Os06g40240,,GO:0006278,Planteome,https://planteome.org/,biological_process.txt
1,tomato,sly,LOC_Os06g40240|LOC_Os06g40240,,LOC_Os06g40240LOC_Os06g40240,,GO:0016032,Planteome,https://planteome.org/,biological_process.txt
2,tomato,sly,LOC_Os07g41220|LOC_Os07g41220,,LOC_Os07g41220LOC_Os07g41220,,GO:0006508,Planteome,https://planteome.org/,biological_process.txt
3,tomato,sly,LOC_Os05g45110|LOC_Os05g45110,,LOC_Os05g45110LOC_Os05g45110,,GO:0008152,Planteome,https://planteome.org/,biological_process.txt
4,tomato,sly,LOC_Os08g30200|LOC_Os08g30200,,LOC_Os08g30200LOC_Os08g30200,,GO:0006333,Planteome,https://planteome.org/,biological_process.txt
5,tomato,sly,LOC_Os08g30200|LOC_Os08g30200,,LOC_Os08g30200LOC_Os08g30200,,GO:0015074,Planteome,https://planteome.org/,biological_process.txt
6,tomato,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0006278,Planteome,https://planteome.org/,biological_process.txt
7,tomato,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0006508,Planteome,https://planteome.org/,biological_process.txt
8,tomato,sly,LOC_Os12g26444|LOC_Os12g26444,,LOC_Os12g26444LOC_Os12g26444,,GO:0015074,Planteome,https://planteome.org/,biological_process.txt
9,tomato,sly,LOC_Os05g43960|LOC_Os05g43960,,LOC_Os05g43960LOC_Os05g43960,,GO:0006355,Planteome,https://planteome.org/,biological_process.txt


In [79]:
df.sample(30)

,species_name,species_code,unique_gene_identifiers,other_gene_identifiers,gene_models,text_unprocessed,annotations,reference_name,reference_link,reference_file
27846,tomato,sly,LOC_Os03g22100|LOC_Os03g22100,,LOC_Os03g22100LOC_Os03g22100,,GO:0008270,Planteome,https://planteome.org/,molecular_function.txt
30662,tomato,sly,LOC_Os01g71910|LOC_Os01g71910,,LOC_Os01g71910LOC_Os01g71910,,GO:0005199,Planteome,https://planteome.org/,molecular_function.txt
34609,tomato,sly,LOC_Os09g21320|LOC_Os09g21320,,LOC_Os09g21320LOC_Os09g21320,,GO:0003723,Planteome,https://planteome.org/,molecular_function.txt
31535,tomato,sly,LOC_Os08g27650|LOC_Os08g27650,,LOC_Os08g27650LOC_Os08g27650,,GO:0050826,Planteome,https://planteome.org/,biological_process.txt
8061,tomato,sly,LOC_Os06g43030|LOC_Os06g43030,,LOC_Os06g43030LOC_Os06g43030,,GO:0006468,Planteome,https://planteome.org/,biological_process.txt
2517,tomato,sly,LOC_Os02g46580|LOC_Os02g46580,,LOC_Os02g46580LOC_Os02g46580,,GO:0006508,Planteome,https://planteome.org/,biological_process.txt
2637,tomato,sly,PURPLE PULVINUS|PU,,,,PO:0007130,Planteome,https://planteome.org/,plant_structural_development_stage.txt
11147,tomato,sly,LOC_Os10g42400|LOC_Os10g42400,,LOC_Os10g42400LOC_Os10g42400,,GO:0005634,Planteome,https://planteome.org/,cellular_component.txt
15371,tomato,sly,LOC_Os11g07270|LOC_Os11g07270,,LOC_Os11g07270LOC_Os11g07270,,GO:0004713,Planteome,https://planteome.org/,molecular_function.txt
54172,tomato,sly,LOC_Os11g14000|LOC_Os11g14000,,LOC_Os11g14000LOC_Os11g14000,,GO:0005543,Planteome,https://planteome.org/,molecular_function.txt
